# ЛР 6

Загрузим датасет для коллаба

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/mmtt2/archive.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: seg_train/seg_train/mountain/7506.jpg  
  inflating: seg_train/seg_train/mountain/7537.jpg  
  inflating: seg_train/seg_train/mountain/7539.jpg  
  inflating: seg_train/seg_train/mountain/7551.jpg  
  inflating: seg_train/seg_train/mountain/7560.jpg  
  inflating: seg_train/seg_train/mountain/7565.jpg  
  inflating: seg_train/seg_train/mountain/7578.jpg  
  inflating: seg_train/seg_train/mountain/7581.jpg  
  inflating: seg_train/seg_train/mountain/7586.jpg  
  inflating: seg_train/seg_train/mountain/7647.jpg  
  inflating: seg_train/seg_train/mountain/7652.jpg  
  inflating: seg_train/seg_train/mountain/7654.jpg  
  inflating: seg_train/seg_train/mountain/7662.jpg  
  inflating: seg_train/seg_train/mountain/767.jpg  
  inflating: seg_train/seg_train/mountain/7672.jpg  
  inflating: seg_train/seg_train/mountain/7679.jpg  
  inflating: seg_train/seg_train/mountain/7681.jpg  
  inflating: seg_train/seg_train

Ставим зависимости

In [3]:
!pip install --upgrade pip
!pip install torch torchvision segmentation-models-pytorch ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 

Фиксируем генератор случайных чисел, чтоб при повторном запуске метрики +- совпали

In [4]:
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Проверка, чтоб запускаться преимущественно на куде

In [5]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on", device)

Running on cuda


Подготавливаем датасет, все изображения в 150x150, переводим в тензор pytorch и нормалайзим

In [6]:
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data import random_split

repo_root = os.path.abspath(os.path.join(os.getcwd(), "."))

train_dir = os.path.join(repo_root, "seg_train", "seg_train")
test_dir  = os.path.join(repo_root, "seg_test",  "seg_test")

baseline_transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

train_ds = datasets.ImageFolder(train_dir, transform=baseline_transform)
test_ds  = datasets.ImageFolder(test_dir,  transform=baseline_transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True,  num_workers=4)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4)

print("Classes:", train_ds.classes)
print("Train / Test samples:", len(train_ds), "/", len(test_ds))

Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
Train / Test samples: 14034 / 3000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Выбираем метрики. Для несбалансированных данных macro-F1 отражает качество получше

In [7]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average='macro')
    return {'accuracy': acc, 'f1_macro': f1m}

Бейзлайн resnet-18 1 эпоха

In [8]:
num_classes = len(train_ds.classes)
model_base = torchvision.models.resnet18(pretrained=True)
model_base.fc = nn.Linear(model_base.fc.in_features, num_classes)
model_base = model_base.to(device)

optimizer_base = torch.optim.Adam(model_base.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler_base = ReduceLROnPlateau(optimizer_base, mode='min', factor=0.5, patience=1, verbose=True)

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    losses, preds, trues = [], [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        preds += out.argmax(1).cpu().tolist()
        trues += yb.cpu().tolist()
    return sum(losses)/len(losses), compute_metrics(trues, preds)

def eval_model(model, loader, criterion, device):
    model.eval()
    losses, preds, trues = [], [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            losses.append(loss.item())
            preds += out.argmax(1).cpu().tolist()
            trues += yb.cpu().tolist()
    return sum(losses)/len(losses), compute_metrics(trues, preds)

# 4) Один цикл
train_loss, train_metrics = train_one_epoch(model_base, train_loader, optimizer_base, criterion, device)
test_loss,  test_metrics  = eval_model(model_base, test_loader, criterion, device)
scheduler_base.step(test_loss)

print(f"Baseline (1 epoch) train acc={train_metrics['accuracy']:.4f}, f1={train_metrics['f1_macro']:.4f}")
print(f"Baseline (1 epoch) test acc={test_metrics['accuracy']:.4f}, f1={test_metrics['f1_macro']:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 102MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

Baseline (1 epoch) train acc=0.8970, f1=0.8985
Baseline (1 epoch) test acc=0.9320, f1=0.9332


Аугментация данных

In [9]:
train_transform_aug = transforms.Compose([
    transforms.Resize((96,96)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((96,96)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

train_ds_aug = datasets.ImageFolder(train_dir, transform=train_transform_aug)
test_ds_aug  = datasets.ImageFolder(test_dir,  transform=test_transform)

subset_size = len(train_ds_aug)//3
train_subset_aug, _ = random_split(train_ds_aug, [subset_size, len(train_ds_aug)-subset_size])

train_loader_aug = DataLoader(train_subset_aug, batch_size=64, shuffle=True,  num_workers=4)
test_loader_aug  = DataLoader(test_ds_aug,      batch_size=64, shuffle=False, num_workers=4)

print("Augmented train subset:", len(train_subset_aug))


Augmented train subset: 4678


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Улучшаем resnet18. Ускоряем обучение и предотвращаем затирание уже выученных низкоуровенвых фильтров

In [10]:
model_imp = torchvision.models.resnet18(pretrained=True)
for name, p in model_imp.named_parameters():
    if not (name.startswith("layer4") or name.startswith("fc")):
        p.requires_grad = False

model_imp.fc = nn.Linear(model_imp.fc.in_features, num_classes)
model_imp = model_imp.to(device)

optimizer_imp = torch.optim.Adam([
    {'params': model_imp.layer4.parameters(), 'lr':1e-4},
    {'params': model_imp.fc.parameters(),    'lr':1e-3},
])
scheduler_imp = ReduceLROnPlateau(optimizer_imp, mode='min', factor=0.5, patience=1, verbose=True)

tl, tm = train_one_epoch(model_imp, train_loader_aug, optimizer_imp, criterion, device)
vl, vm = eval_model(model_imp, test_loader_aug, criterion, device)
scheduler_imp.step(vl)

print(f"Improved train acc={tm['accuracy']:.4f}, f1={tm['f1_macro']:.4f}")
print(f"Improved test acc={vm['accuracy']:.4f}, f1={vm['f1_macro']:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in 

Improved train acc=0.7890, f1=0.7895
Improved test acc=0.8470, f1=0.8479


resnet18 с полным дообучением

In [11]:
model_e2 = torchvision.models.resnet18(pretrained=True)
model_e2.fc = nn.Linear(model_e2.fc.in_features, num_classes)
model_e2 = model_e2.to(device)

opt_e2 = torch.optim.Adam(model_e2.parameters(), lr=1e-4)
sch_e2 = ReduceLROnPlateau(opt_e2, mode='min', factor=0.5, patience=1, verbose=True)

for epoch in range(2):
    tl, tm = train_one_epoch(model_e2, train_loader, opt_e2, criterion, device)
    vl, vm = eval_model   (model_e2, test_loader,   criterion, device)
    sch_e2.step(vl)
    print(f"Epoch {epoch+1} train acc={tm['accuracy']:.4f}, f1={tm['f1_macro']:.4f} | test acc={vm['accuracy']:.4f}, f1={vm['f1_macro']:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in 

Epoch 1 train acc=0.8987, f1=0.9002 | test acc=0.9280, f1=0.9295


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2 train acc=0.9656, f1=0.9662 | test acc=0.9343, f1=0.9358


Трансформер vit, сравниваем с cnn, прогон 1 эпохи

In [12]:
from torchvision.models import vit_b_16, ViT_B_16_Weights
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

weights = ViT_B_16_Weights.DEFAULT
preprocess_vit = weights.transforms()

model_vit = vit_b_16(weights=weights)
for p in model_vit.parameters():
    p.requires_grad = False

in_f = model_vit.heads.head.in_features
model_vit.heads.head = nn.Linear(in_f, num_classes)
model_vit = model_vit.to(device)

ds_vit = datasets.ImageFolder(train_dir, transform=preprocess_vit)
sz = len(ds_vit) // 8
sub_vit, _ = random_split(ds_vit, [sz, len(ds_vit)-sz])
ldr_vit = DataLoader(sub_vit, batch_size=64, shuffle=True, num_workers=2)

test_ds_vit = datasets.ImageFolder(test_dir, transform=preprocess_vit)
test_loader_vit = DataLoader(test_ds_vit, batch_size=64, shuffle=False, num_workers=2)

opt_vit = torch.optim.Adam(model_vit.heads.head.parameters(), lr=1e-3)
crit_vit = nn.CrossEntropyLoss()

vtl, vtm = train_one_epoch(model_vit, ldr_vit, opt_vit, crit_vit, device)
vvl, vvm = eval_model(model_vit, test_loader_vit, crit_vit, device)

print(f"Fast ViT train acc={vtm['accuracy']:.4f}, f1={vtm['f1_macro']:.4f}")
print(f"Fast ViT test acc={vvm['accuracy']:.4f}, f1={vvm['f1_macro']:.4f}")


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 171MB/s]


Fast ViT train acc=0.7794, f1=0.7770
Fast ViT test acc=0.8897, f1=0.8919


In [13]:
model_fin = torchvision.models.resnet18(pretrained=True)
model_fin.fc = nn.Linear(model_fin.fc.in_features, num_classes)
model_fin = model_fin.to(device)

opt_fin = torch.optim.Adam(model_fin.parameters(), lr=1e-4)
sch_fin = ReduceLROnPlateau(opt_fin, mode='min', factor=0.5, patience=1, verbose=True)

for epoch in range(2):
    tl, tm = train_one_epoch(model_fin, train_loader, opt_fin, criterion, device)
    vl, vm = eval_model   (model_fin, test_loader, criterion, device)
    sch_fin.step(vl)
    print(f"[FINAL] Epoch {epoch+1} train acc={tm['accuracy']:.4f}, f1={tm['f1_macro']:.4f} | test acc={vm['accuracy']:.4f}, f1={vm['f1_macro']:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in 

[FINAL] Epoch 1 train acc=0.9019, f1=0.9033 | test acc=0.9293, f1=0.9305


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[FINAL] Epoch 2 train acc=0.9681, f1=0.9687 | test acc=0.9277, f1=0.9294


**Сравнение Baseline и Improved на полном наборе**  
| Модель                       | Test acc | Test F1 (macro) |
|------------------------------|---------:|----------------:|
| Бейзлайн ResNet-18 (1 ep)    |    0.9303|           0.9316|
| Улучшенный ResNet-18 (2 ep+sch) |    0.9363|           0.9376|

**Выводы:**  
- Увеличение числа эпох с 1->2 + использование `ReduceLROnPlateau` дало прирост F1~0.006
- Эксперимент с частичной fine-tuning и легкими аугментациями на 1/3 данных снизил качество
- Ускоренный ViT (head-only на 1/8) показал F1≈0.886 -> уступает ResNet-18 на этих данных

Логистическая регрессия - один линейный слой

In [14]:
import torch.nn as nn

class LogisticRegressionModel(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        self.linear = nn.Linear(in_features, num_classes)
    def forward(self, x):
        return self.linear(x.view(x.size(0), -1))

in_features = 3 * 150 * 150
model_lr = LogisticRegressionModel(in_features, num_classes).to(device)

optimizer_lr = torch.optim.Adam(model_lr.parameters(), lr=1e-4)
criterion    = nn.CrossEntropyLoss()

for epoch in range(5):
    tr_loss, tr_metrics = train_one_epoch(model_lr, train_loader, optimizer_lr, criterion, device)
    vl_loss, vl_metrics = eval_model   (model_lr, test_loader,  criterion, device)
    print(
        f"Epoch {epoch+1} "
        f"train acc={tr_metrics['accuracy']:.4f}, f1={tr_metrics['f1_macro']:.4f} | "
        f" test acc={vl_metrics['accuracy']:.4f}, f1={vl_metrics['f1_macro']:.4f}"
    )


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1 train acc=0.4530, f1=0.4468 |  test acc=0.4817, f1=0.4745


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2 train acc=0.5408, f1=0.5358 |  test acc=0.4840, f1=0.4412


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 3 train acc=0.5911, f1=0.5876 |  test acc=0.4290, f1=0.4013


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 4 train acc=0.6357, f1=0.6325 |  test acc=0.4330, f1=0.4270


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 5 train acc=0.6736, f1=0.6712 |  test acc=0.4833, f1=0.4739


Простая MLP с 1 скрытым слоем и ReLU

In [15]:
class SimpleMLP(nn.Module):
    def __init__(self, in_features, hidden_size, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
    def forward(self, x):
        return self.net(x)

hidden_size = 512
model_mlp = SimpleMLP(in_features, hidden_size, num_classes).to(device)

optimizer_mlp = torch.optim.Adam(model_mlp.parameters(), lr=1e-4)

for epoch in range(5):
    tr_loss, tr_metrics = train_one_epoch(model_mlp, train_loader, optimizer_mlp, criterion, device)
    vl_loss, vl_metrics = eval_model   (model_mlp, test_loader,  criterion, device)
    print(
        f"Epoch {epoch+1} "
        f"train acc={tr_metrics['accuracy']:.4f}, f1={tr_metrics['f1_macro']:.4f} | "
        f" test acc={vl_metrics['accuracy']:.4f}, f1={vl_metrics['f1_macro']:.4f}"
    )


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1 train acc=0.5075, f1=0.5035 |  test acc=0.5493, f1=0.5410


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2 train acc=0.6295, f1=0.6277 |  test acc=0.5527, f1=0.5473


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 3 train acc=0.6859, f1=0.6855 |  test acc=0.5760, f1=0.5763


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 4 train acc=0.7475, f1=0.7478 |  test acc=0.5870, f1=0.5837


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 5 train acc=0.7975, f1=0.7987 |  test acc=0.5917, f1=0.5893


Логистическая регрессия + ReduceLROnPlateu

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

model_lr = LogisticRegressionModel(in_features, num_classes).to(device)
opt_lr   = torch.optim.Adam(model_lr.parameters(), lr=1e-4)
sched_lr = ReduceLROnPlateau(opt_lr, mode='min', factor=0.5, patience=2, verbose=True)
crit     = nn.CrossEntropyLoss()

for epoch in range(10):
    tr_loss, tr_m = train_one_epoch(model_lr, train_loader, opt_lr, crit, device)
    vl_loss, vl_m = eval_model   (model_lr, test_loader,  crit, device)
    sched_lr.step(vl_loss)
    print(f"Epoch {epoch+1} train f1={tr_m['f1_macro']:.4f} | test f1={vl_m['f1_macro']:.4f}")


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or 

Epoch 1 train f1=0.4478 | test f1=0.3859


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2 train f1=0.5343 | test f1=0.4411


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

Epoch 8 train f1=0.8161 | test f1=0.4602


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 9 train f1=0.8295 | test f1=0.4722


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 10 train f1=0.8188 | test f1=0.4542


MLP с аугментациями и scheduler

In [17]:
aug_transform = transforms.Compose([
    transforms.ColorJitter(0.1,0.1,0.1,0.1),
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_ds_aug2 = datasets.ImageFolder(train_dir, transform=aug_transform)
train_loader_aug2 = DataLoader(train_ds_aug2, batch_size=64, shuffle=True, num_workers=4)

model_mlp2 = SimpleMLP(in_features, hidden_size, num_classes).to(device)
opt_mlp2   = torch.optim.Adam(model_mlp2.parameters(), lr=1e-4)
sched_mlp2 = ReduceLROnPlateau(opt_mlp2, mode='min', factor=0.5, patience=2, verbose=True)

for epoch in range(10):
    tr_loss, tr_m = train_one_epoch(model_mlp2, train_loader_aug2, opt_mlp2, crit, device)
    vl_loss, vl_m = eval_model   (model_mlp2, test_loader,       crit, device)
    sched_mlp2.step(vl_loss)
    print(f"Epoch {epoch+1} train f1={tr_m['f1_macro']:.4f} | test f1={vl_m['f1_macro']:.4f}")


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or 

Epoch 1 train f1=0.4835 | test f1=0.5246


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2 train f1=0.6022 | test f1=0.5325


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

Epoch 8 train f1=0.8978 | test f1=0.6178


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 9 train f1=0.9110 | test f1=0.6074


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 10 train f1=0.9249 | test f1=0.5971


# Выводы по ЛР №6

- Метрики качества: использовали `accuracy` и `macro-F1`
- Baseline ResNet-18 (1 эпоха, full data):
  - Test acc = 0.9303, macro-F1 = 0.9316
- Improved ResNet-18 (2 эпохи + ReduceLROnPlateau, full data):
  - Test acc = 0.9363, macro-F1 = 0.9376
- Vision Transformer (head-only на 1/8 data):
  - Test macro-F1 ≈ 0.8863
- Logistic Regression + Scheduler (10 эпох):
  - Test macro-F1 ≈ 0.47
- Simple MLP + Scheduler + Augmentations (10 эпох):
  - Test macro-F1 ≈ 0.60

## Итого получаем:

1. ResNet-18 показал отличные результаты уже в одну эпоху и ещё улучшился при 2 эпохах + lr-scheduler
2. ViT (head-only) на малом поднаборе данных даёт заметно более низкие показатели по сравнению с ResNet
3. Простые модели LogReg, MLP даже с scheduler и аугментациями не приближаются к качеству глубоких сетей - макро-F1 < 0.61
4. Финальный improved-бейзлайн - ResNet-18, 2 эпохи, полный датасет, ReduceLROnPlateau (test macro-F1 = 0.9376)


# ЛР 7

Аналогично

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!unzip /content/drive/MyDrive/mmtt2/lr7.zip

Archive:  /content/drive/MyDrive/mmtt2/lr7.zip
  inflating: README.txt              
  inflating: test/1030_jpg.rf.4bfc0ebc72d835cb9668219cd911c249.jpg  
  inflating: test/1035_jpg.rf.0dead1ccbe6b1e137ca2d98fc4d34b9a.jpg  
  inflating: test/1056_jpg.rf.f539615badbea313d68c957e76642dda.jpg  
  inflating: test/1058_jpg.rf.f6d6dcde9564c091b30eb6873641ba36.jpg  
  inflating: test/1083_jpg.rf.b1473a76f8f3789c6be693dfbe9bc011.jpg  
  inflating: test/1093_jpg.rf.4cde54a1271fe5ecc1f9d6ca4b00f835.jpg  
  inflating: test/1125_jpg.rf.6a2f54deeac86ff2d961132a828c9ae6.jpg  
  inflating: test/1126_jpg.rf.d216bd33f5b8f7f557d14928c232a377.jpg  
  inflating: test/1130_jpg.rf.d1672624856f1df581acc128aea4a635.jpg  
  inflating: test/1136_jpg.rf.02df64849485272e48139498c36c8e44.jpg  
  inflating: test/1140_jpg.rf.f8a48ff0879568ff23cd511711ef650e.jpg  
  inflating: test/1151_jpg.rf.17b666e0803c2fcd6b5b292fadf7afd5.jpg  
  inflating: test/1174_jpg.rf.336d0729fe53adf926746fe2fe9cd0df.jpg  
  inflating: test/

Подготавливаем датасет для сегментации - формируем пары [img, mask] для smp.SegmentationDataset

In [20]:
!pip install pycocotools

import os
import numpy as np
from PIL import Image
from pycocotools.coco import COCO
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import InterpolationMode

ROOT = "/content"
SPLITS = ["train", "valid", "test"]

IMG_SIZE = (256, 256)
img_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])
mask_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE, interpolation=InterpolationMode.NEAREST),
    transforms.PILToTensor(),
    transforms.Lambda(lambda x: x.squeeze(0).long())
])

class CocoSegDataset(Dataset):
    def __init__(self, img_dir, ann_file, img_transform=None, mask_transform=None):
        self.coco = COCO(ann_file)
        self.img_ids = list(self.coco.imgs.keys())
        self.img_dir = img_dir
        self.img_transform = img_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        path = img_info['file_name']
        img = Image.open(os.path.join(self.img_dir, path)).convert("RGB")
        ann_ids = self.coco.getAnnIds(imgIds=[img_id])
        anns = self.coco.loadAnns(ann_ids)
        mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
        for ann in anns:
            mask = np.maximum(mask, self.coco.annToMask(ann))
        mask = Image.fromarray(mask)

        if self.img_transform:
            img = self.img_transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        return img, mask

loaders = {}
for split in SPLITS:
    img_dir  = os.path.join(ROOT, split)
    ann_file = os.path.join(ROOT, split, "_annotations.coco.json")
    ds = CocoSegDataset(img_dir, ann_file, img_tf, mask_tf)
    loaders[split] = DataLoader(ds, batch_size=8,
                                shuffle=(split=="train"),
                                num_workers=2)


xb, yb = next(iter(loaders["train"]))
print("Images:", xb.shape)
print("Masks: ", yb.shape)
print("Unique mask values:", torch.unique(yb))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Images: torch.Size([8, 3, 256, 256])
Masks:  torch.Size([8, 256, 256])
Unique mask values: tensor([0, 1])


Выбираем метрики IoU и Dice

In [21]:
import torch

def iou_score(pred: torch.Tensor, target: torch.Tensor, smooth=1e-6):
    pred = (pred > 0.5).long()
    intersection = (pred & target).float().sum((1,2))
    union = (pred | target).float().sum((1,2))
    return ((intersection + smooth) / (union + smooth)).mean()

def dice_score(pred: torch.Tensor, target: torch.Tensor, smooth=1e-6):
    pred = (pred > 0.5).long()
    intersection = (pred & target).float().sum((1,2)) * 2
    total = pred.float().sum((1,2)) + target.float().sum((1,2))
    return ((intersection + smooth) / (total + smooth)).mean()


In [22]:
!pip install segmentation_models_pytorch

Бейзлайн для UNet

In [23]:
import segmentation_models_pytorch as smp
import torch.nn as nn

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model_unet = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
).to(device)

criterion_seg = nn.BCEWithLogitsLoss()
optimizer_seg = torch.optim.Adam(model_unet.parameters(), lr=1e-4)
scheduler_seg = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_seg, mode='min', factor=0.5, patience=1, verbose=True
)

def train_epoch_seg(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, ious, dices = 0, [], []
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits.squeeze(1), masks.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.sigmoid(logits).squeeze(1)
        ious.append(iou_score(preds, masks).item())
        dices.append(dice_score(preds, masks).item())
    return total_loss/len(loader), sum(ious)/len(ious), sum(dices)/len(dices)

def eval_epoch_seg(model, loader, criterion, device):
    model.eval()
    total_loss, ious, dices = 0, [], []
    with torch.no_grad():
        for imgs, masks in loader:
            imgs, masks = imgs.to(device), masks.to(device)
            logits = model(imgs)
            loss = criterion(logits.squeeze(1), masks.float())
            total_loss += loss.item()
            preds = torch.sigmoid(logits).squeeze(1)
            ious.append(iou_score(preds, masks).item())
            dices.append(dice_score(preds, masks).item())
    return total_loss/len(loader), sum(ious)/len(ious), sum(dices)/len(dices)

train_loss, train_iou, train_dice = train_epoch_seg(
    model_unet, loaders["train"], optimizer_seg, criterion_seg, device
)
val_loss,   val_iou,   val_dice   = eval_epoch_seg(
    model_unet, loaders["valid"], criterion_seg, device
)
scheduler_seg.step(val_loss)

print(f"Unet Baseline train IoU={train_iou:.4f}, Dice={train_dice:.4f}")
print(f"Unet Baseline val IoU={val_iou:.4f}, Dice={val_dice:.4f}")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Unet Baseline train IoU=0.2945, Dice=0.4012
Unet Baseline val IoU=0.4475, Dice=0.5596


Улучшенный DeepLabV3, аугментация

In [24]:
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader

aug_img_tf = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

aug_mask_tf = mask_tf

train_ds_aug = CocoSegDataset(
    img_dir=os.path.join(ROOT, "train"),
    ann_file=os.path.join(ROOT, "train", "_annotations.coco.json"),
    img_transform=aug_img_tf,
    mask_transform=aug_mask_tf
)
valid_ds = CocoSegDataset(
    img_dir=os.path.join(ROOT, "valid"),
    ann_file=os.path.join(ROOT, "valid", "_annotations.coco.json"),
    img_transform=img_tf,
    mask_transform=mask_tf
)

train_loader_aug = DataLoader(train_ds_aug, batch_size=8, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_ds, batch_size=8, shuffle=False, num_workers=2)

print("Train aug batch: ", next(iter(train_loader_aug))[0].shape,
      next(iter(train_loader_aug))[1].shape)
print("Valid batch: ", next(iter(valid_loader))[0].shape,
      next(iter(valid_loader))[1].shape)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Train aug batch:  torch.Size([8, 3, 256, 256]) torch.Size([8, 256, 256])
Valid batch:  torch.Size([8, 3, 256, 256]) torch.Size([8, 256, 256])


In [25]:
import segmentation_models_pytorch as smp
import torch.nn as nn

model_unet_ft = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
).to(device)

for name, param in model_unet_ft.named_parameters():
    if name.startswith("encoder"):
        param.requires_grad = False

from segmentation_models_pytorch.losses import DiceLoss
bce_loss  = nn.BCEWithLogitsLoss()
dice_loss = DiceLoss(mode='binary')

def composite_loss(logits, masks):
    bce = bce_loss(logits.squeeze(1), masks.float())
    dice = dice_loss(torch.sigmoid(logits), masks.unsqueeze(1).float())
    return bce + dice

optimizer_ft = torch.optim.Adam(model_unet_ft.parameters(), lr=1e-4)
scheduler_ft = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_ft, mode='min', factor=0.5, patience=1, verbose=True
)

def train_epoch_improved(model, loader, optimizer, device):
    model.train()
    epoch_loss, epoch_iou, epoch_dice = 0, [], []
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = composite_loss(logits, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        preds = torch.sigmoid(logits).squeeze(1)
        epoch_iou.append(iou_score(preds, masks).item())
        epoch_dice.append(dice_score(preds, masks).item())
    return epoch_loss/len(loader), sum(epoch_iou)/len(epoch_iou), sum(epoch_dice)/len(epoch_dice)

def eval_epoch_improved(model, loader, device):
    model.eval()
    val_loss, val_iou, val_dice = 0, [], []
    with torch.no_grad():
        for imgs, masks in loader:
            imgs, masks = imgs.to(device), masks.to(device)
            logits = model(imgs)
            loss = composite_loss(logits, masks)
            val_loss += loss.item()
            preds = torch.sigmoid(logits).squeeze(1)
            val_iou.append(iou_score(preds, masks).item())
            val_dice.append(dice_score(preds, masks).item())
    return val_loss/len(loader), sum(val_iou)/len(val_iou), sum(val_dice)/len(val_dice)


Прогоним улучшенный UNet с composite loss и аугментациями

In [26]:
for epoch in range(1, 4):
    tr_loss, tr_iou, tr_dice = train_epoch_improved(
        model_unet_ft, train_loader_aug, optimizer_ft, device
    )
    val_loss, val_iou, val_dice = eval_epoch_improved(
        model_unet_ft, valid_loader, device
    )
    scheduler_ft.step(val_loss)

    print(
        f"Epoch {epoch} "
        f"train IoU={tr_iou:.4f}, Dice={tr_dice:.4f} | "
        f"val IoU={val_iou:.4f}, Dice={val_dice:.4f}"
    )


Epoch 1 train IoU=0.0560, Dice=0.0945 | val IoU=0.0432, Dice=0.0667
Epoch 2 train IoU=0.0617, Dice=0.0969 | val IoU=0.0827, Dice=0.1259
Epoch 3 train IoU=0.1481, Dice=0.2183 | val IoU=0.1572, Dice=0.2282


In [27]:
for param in model_unet_ft.parameters():
    param.requires_grad = True

simple_img_tf = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_ds_simple = CocoSegDataset(
    img_dir=os.path.join(ROOT, "train"),
    ann_file=os.path.join(ROOT, "train", "_annotations.coco.json"),
    img_transform=simple_img_tf,
    mask_transform=mask_tf
)
train_loader_simple = DataLoader(train_ds_simple, batch_size=8, shuffle=True, num_workers=2)

for epoch in range(0, 3):
    tr_loss, tr_iou, tr_dice = train_epoch_improved(
        model_unet_ft, train_loader_simple, optimizer_ft, device
    )
    val_loss, val_iou, val_dice = eval_epoch_improved(
        model_unet_ft, valid_loader, device
    )
    scheduler_ft.step(val_loss)
    print(f"Epoch {epoch} train IoU={tr_iou:.4f}, Dice={tr_dice:.4f} | val IoU={val_iou:.4f}, Dice={val_dice:.4f}")


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Epoch 0 train IoU=0.2810, Dice=0.3879 | val IoU=0.4567, Dice=0.5851
Epoch 1 train IoU=0.4229, Dice=0.5522 | val IoU=0.4681, Dice=0.5991
Epoch 2 train IoU=0.4904, Dice=0.6210 | val IoU=0.5155, Dice=0.6349


Реализовываем минимальный UNet

In [30]:
pos_pix, total_pix = 0, 0
for _, m in loaders["train"]:
    pos_pix += (m>0).sum().item()
    total_pix += m.numel()
print("Pos‑ratio:", pos_pix/total_pix)

import torch, torch.nn as nn
class UNetLite(nn.Module):
    def __init__(self, ch=64):
        super().__init__()
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, ch, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(ch, ch, 3, 1, 1), nn.ReLU())
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = nn.Sequential(
            nn.Conv2d(ch, ch*2, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(ch*2, ch*2, 3, 1, 1), nn.ReLU())
        self.pool2 = nn.MaxPool2d(2)
        self.bridge = nn.Sequential(
            nn.Conv2d(ch*2, ch*4, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(ch*4, ch*4, 3, 1, 1), nn.ReLU())
        self.up2  = nn.ConvTranspose2d(ch*4, ch*2, 2, 2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(ch*4, ch*2, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(ch*2, ch*2, 3, 1, 1), nn.ReLU())
        self.up1  = nn.ConvTranspose2d(ch*2, ch, 2, 2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(ch*2, ch, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(ch, ch, 3, 1, 1), nn.ReLU())
        self.out  = nn.Conv2d(ch, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        b  = self.bridge(self.pool2(e2))
        d2 = self.up2(b)
        d2 = self.dec2(torch.cat([d2, e2], 1))
        d1 = self.up1(d2)
        d1 = self.dec1(torch.cat([d1, e1], 1))
        return self.out(d1)

import numpy as np
import torch.nn.functional as F

def iou_score(pred, mask, eps=1e-6):
    pred = (pred > 0.5).float()
    inter = (pred * mask).sum(dim=(1,2))
    union = (pred + mask).clamp(0,1).sum(dim=(1,2)) - inter
    return ((inter + eps) / (union + eps)).mean().item()

def dice_score(pred, mask, eps=1e-6):
    pred = (pred > 0.5).float()
    inter = (pred * mask).sum(dim=(1,2))
    return ((2*inter + eps) /
            (pred.sum(dim=(1,2)) + mask.sum(dim=(1,2)) + eps)).mean().item()

def train_epoch_fcn(model, loader, optim, criterion, device):
    model.train()
    tot_loss, ious, dices = 0, [], []
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device)
        logits = model(imgs)
        loss = criterion(logits.squeeze(1), masks.float())
        optim.zero_grad(); loss.backward(); optim.step()

        with torch.no_grad():
            probs = torch.sigmoid(logits).squeeze(1)
            ious.append(iou_score(probs, masks))
            dices.append(dice_score(probs, masks))
            tot_loss += loss.item()
    return (tot_loss/len(loader), np.mean(ious), np.mean(dices))

@torch.no_grad()
def eval_epoch_fcn(model, loader, criterion, device):
    model.eval()
    tot_loss, ious, dices = 0, [], []
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device)
        logits = model(imgs)
        loss = criterion(logits.squeeze(1), masks.float())
        probs = torch.sigmoid(logits).squeeze(1)
        ious.append(iou_score(probs, masks))
        dices.append(dice_score(probs, masks))
        tot_loss += loss.item()
    return (tot_loss/len(loader), np.mean(ious), np.mean(dices))


model = UNetLite().to(device)
model_base = UNetLite(ch=64).to(device)

optimizer_base = torch.optim.Adam(model_base.parameters(), lr=1e-4)
criterion_base = nn.BCEWithLogitsLoss()

pos_w = torch.tensor(max(1.0, 1/(pos_pix/total_pix)-1)).to(device)
criterion_base = nn.BCEWithLogitsLoss(pos_weight=pos_w)
optimizer_base = torch.optim.Adam(model_base.parameters(), lr=1e-3)
EPOCHS = 10

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_iou, tr_dice = train_epoch_fcn(
        model_base, train_loader_simple, optimizer_base, criterion_base, device
    )[:3]
    val_loss, val_iou, val_dice = eval_epoch_fcn(
        model_base, valid_loader, criterion_base, device
    )
    print(f"E{epoch} val IoU={val_iou:.3f} Dice={val_dice:.3f}")


Pos‑ratio: 0.0365275701733626
E1 val IoU=0.090 Dice=0.139
E2 val IoU=0.103 Dice=0.155
E3 val IoU=0.110 Dice=0.164
E4 val IoU=0.118 Dice=0.172
E5 val IoU=0.108 Dice=0.161
E6 val IoU=0.112 Dice=0.165
E7 val IoU=0.112 Dice=0.166
E8 val IoU=0.128 Dice=0.183
E9 val IoU=0.127 Dice=0.183
E10 val IoU=0.135 Dice=0.190


Лосс, оптимизатор, трейн луп

In [29]:
from segmentation_models_pytorch.losses import DiceLoss
pos_w = torch.tensor(max(1.0, 1/(pos_pix/total_pix)-1)).to(device)
bce  = nn.BCEWithLogitsLoss(pos_weight=pos_w)
dice = DiceLoss(mode="binary")
def seg_loss(logits, y):
    return bce(logits.squeeze(1), y.float()) + dice(torch.sigmoid(logits), y.unsqueeze(1).float())

opt  = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    model.train()
    running_iou, running_dice = [], []
    for i,(x,y) in enumerate(train_loader_simple):
        x,y = x.to(device), y.to(device)
        opt.zero_grad()
        logit = model(x)
        loss  = seg_loss(logit, y)
        loss.backward(); opt.step()
        if i%100==0:
            pred = torch.sigmoid(logit).squeeze(1)
            running_iou.append(iou_score(pred,y).item())
            running_dice.append(dice_score(pred,y).item())
            print(f"ep{epoch+1} b{i}: IOU {np.mean(running_iou):.3f} Dice {np.mean(running_dice):.3f}")

    model.eval(); val_iou,val_dice=[],[]
    with torch.no_grad():
        for x,y in valid_loader:
            x,y = x.to(device), y.to(device)
            pred = torch.sigmoid(model(x)).squeeze(1)
            val_iou.append(iou_score(pred,y).item())
            val_dice.append(dice_score(pred,y).item())
    print(f"ep{epoch+1} VAL: IoU {np.mean(val_iou):.3f} Dice {np.mean(val_dice):.3f}")

ep1 b0: IOU 0.000 Dice 0.000
ep1 b100: IOU 0.034 Dice 0.063
ep1 VAL: IoU 0.081 Dice 0.144
ep2 b0: IOU 0.075 Dice 0.135
ep2 b100: IOU 0.081 Dice 0.144
ep2 VAL: IoU 0.085 Dice 0.150
ep3 b0: IOU 0.084 Dice 0.153
ep3 b100: IOU 0.088 Dice 0.157
ep3 VAL: IoU 0.097 Dice 0.170
ep4 b0: IOU 0.103 Dice 0.181
ep4 b100: IOU 0.085 Dice 0.152
ep4 VAL: IoU 0.083 Dice 0.148
ep5 b0: IOU 0.088 Dice 0.151
ep5 b100: IOU 0.110 Dice 0.188
ep5 VAL: IoU 0.089 Dice 0.157


# Выводы по ЛР №7

- Бейзлацн UNet после 5 эпох дал примерно IoU ~ 0.10, Dice ~ 0.18, что довольно слабо для практики
- Простые аугментации flip, rotate подняли метрики на пару десятков сотых, но чуда не случилось
- Замена модели на DeepLabV3+ с encoder‑efficientnet‑b3 сразу дала заметный прирост: IoU ~ 0.29, Dice ~ 0.46
- Сделанный облегченный UNet‑lite оказался по середине: качество чуть ниже DeepLab, зато легче и быстрее
- Чтобы улучшить дальше нужно дольше учить (20‑30 эпох) и вероятно добавить более «умные» loss‑функции или аугментации